# Descriptive Statistics Review

## Before you start:

- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

## Context

![img](./diamonds.jpg)

The dataset we will be using is comprised of approximately 54k rows and 11 different columns. As always a row represents a single observation (in this case a diamond) and each of the columns represent a different feature of a diamond.

In this lab we will first explore our dataset to better understand the data we have and then try to determine which characteristics of a diamond are more likely to influence its price.

The following codebook was provided together with the dataset to clarify what each column represents:

**price:** price in US dollars (326-18,823)

**carat:** weight of the diamond (0.2--5.01)

**cut:** quality of the cut (Fair, Good, Very Good, Premium, Ideal)

**color:** diamond colour, from J (worst) to D (best)

**clarity:** a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))

**x:** length in mm (0--10.74)

**y:** width in mm (0--58.9)

**z:** depth in mm (0--31.8)

**depth:** total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)

**table:** width of top of diamond relative to widest point (43--95)

| Column  | Description  |
|---|---|
| Price  | Price in US dollars (326-18,823)  |
| Carat  | Weight of the diamond (0.2--5.01)  |
| Cut  | Quality of the cut (Fair, Good, Very Good, Premium, Ideal)  |
| Color  | Diamond colour, from J (worst) to D (best)  |
| Clarity  | A measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))   |
| x  | Length in mm (0--10.74)  |
| y  | Width in mm (0--58.9)  |
| z  | Depth in mm (0--31.8)  |
| Depth  | Total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)  |
| Table  | Width of top of diamond relative to widest point (43--95)  |

## Libraries
Pandas and numpy will be needed for the analysis of the data. Don't worry about the seaborn and matplotlib import at the moment, you will learn more about them in the following week.

In [1]:
- (2*8) * 3 / (2*8)

-3.0

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

First import the data from the .csv file provided and assign it to a variable named diamonds.

In [ ]:
diamonds = pd.read_csv('diamonds.csv')

In [ ]:
diamonds = diamonds.drop('Unnamed: 0', axis=1)

Let's see how the data looks like by using pandas methods like head(), sample(), and describe().

In [ ]:
diamonds.head()

In [ ]:
diamonds.describe()

You have probably noticed that the columns x, y and z have a minimum value of 0. This means that there are one or more rows (or observations) in our dataset that are supposedly representing a diamond that has lenght, width or depth of 0 which, considering that we're talking about a physical object, is impossible!

We will proceed to check the rows that have a value of 0 in any of the x, y or z columns. By doing this we want to check if the data we're missing can be obtained using the data that we do have.

In [ ]:
diamonds.query('x== 0 or z == 0 or y == 0')

As you can see, we have 20 rows that have a value of 0 in some or all the aforementioned columns.
Most of them (12) are missing the z value, which we can obtain using the columns depth, x and y. For these rows, we will create a function that applies the formula given in the codebook and get the value of z. We will drop the other rows (8), since they are missing all 3 values or 2 of them.

In [ ]:
def get_z(row):
    """
    Input: pd.Series
    Output: Int
    
    This function takes depth, x and y and calculates the value of z.
    """
    x = row['x']
    y = row['y']
    depth = row['depth']

    return (np.mean([x,y]) * depth) / 100 if row['z'] == 0 else row['z']

In [ ]:
diamonds['z'] = diamonds.apply(get_z, axis=1)

If we leave the other 8 values as they are it'd negatively affect our analysis and therefore it is better to consider those values as NaN values, since they are probably the result of a mistake or error during the whole process going from measuring those values to storing them in a dataset.

To replace them we can use the pandas .replace() method and np.NaN.

In [ ]:
diamonds[['x','y','z']] = diamonds[['x','y','z']].replace(0, np.NaN)

Let's check the data again with describe().

In [ ]:
diamonds.describe()

Good! The minimum value for x, y and z is now a positive number, as it should be for the physical measures of an object.

Another good practice is to check for NaN values in the dataset. Since we introduce them ourselves with the replace we just applied we will surely find some, but there may be more that are unrelated to the x, y and z columns. Checking NaNs is a fundamental part of data cleaning and it's always better to do this kind of operations before proceeding with the actual analysis.

In [ ]:
diamonds.isna().sum()

Since we only have 8 null values over a total of 54000 observations and we can't have a way of replacing the missing values, we can safely drop the related rows since they represent a minimal part of our dataset. There could be cases where you have some columns that have a huge number of NaNs. In those cases it is always better to try and understand the reason for that before dropping the rows, because doing that may mean that you are removing a relevant portion of your data.

In [ ]:
diamonds = diamonds.dropna()

Now we're going to revisit the summary table and check for outliers.

In [ ]:
diamonds.describe()

We can clearly see that the price and x, y and z columns have some weird values if we check the max row. Let's look further into this.

We're going to filter our DataFrame, we're going to take all the values that have a price higher than the 75th percentile. EVEN CHECKING WITH THE 99 QUANTILE WE HAVE DIFFICULTIES FINDING THE OUTLIERS. MAYBE CHECKING FOR VALUES HIGHER THAN A NUMBER?

In [ ]:
quantile90 = diamonds.y.quantile(0.99)
diamonds[diamonds.y > quantile90]

In [ ]:
diamonds[diamonds.z > 10]

WE ALREADY HAVE A FORMULA TO CALCULATE Z, SO WE JUST NEED TO APPLY IT HERE TO FIX THIS OUTLIER

In [ ]:
diamonds[diamonds.y > 10]

### FORMULA TO CALCULATE Y ----> Y = (2Z - X*DEPTH) / DEPTH

In [ ]:
def get_y(row):
    x = row['x']
    y = row['y']
    z = row['depth']
    
    return  (2 * z - x * depth ) / depth

WE CAN CLEARLY SEE THAT THE 31.8 IS AN OUTLIER FOR THE Y VALUE. AND WE CAN SEE THAT THE 58.9 IS THE DEPTH, SO IT WAS A MISTAKE WHEN THEY INTRODUCED THE DATA. SHOULD WE APPLY THE FORMULA TO RECOVER Y IN BOTH CASES?

The clarity column is what's called a categorical variable: it means that it represents a qualitative property and therefore can only take a discrete number of possible values. Given that the current values (like VVS2 and SI1) are easily understood let's replace them with a numerical scale, where 0 represent the lowest clarity and 7 the highest.

In [ ]:
clarity = {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}
for i, j in clarity.items():
    diamonds.clarity.replace(i, j, inplace=True)

Now that we have cleaned our data we can proceed with some exploratory analysis. Let's start by looking at how the charateristics of a diamond (especially the price, since that's our focus) change based on its color. Remember that you can use the groupby() method in pandas. 

Let's check the max, min and the mean.

In [ ]:
diamonds.groupby('color').max()

In [ ]:
diamonds.groupby('color').min()

In [ ]:
diamonds.groupby('color').mean()

As you can see most features of a diamond do not change much based on the color apart from the price and the x, y and z dimensions.

Let's plot the frequenc distribution of the diamonds color in our dataset. Remember that you can use the pandas plot() method.

In [ ]:
diamonds['color'].value_counts().plot(kind='bar')

Here we will ask the students to revisit the overview of the dataset. We will ask the to comment on the measure of variability.
We can see that there is little variation between the variables (since we're dealing with such a small scale this is normal). But we can see that there's a huge variation when we're dealing with the price variable. Thus we can say that even small changes between diamons can affect the price.

In [ ]:
diamonds.describe()

We will give them the code to plot a heatmap and a correlation matrix, since they don't know how to plot and they were asked already to create a function to calculate correlation and covariance.

We'll ask them to comment on the correlation matrix.

In [ ]:
sns.pairplot(diamonds)

In [ ]:
plt.figure(figsize=(20, 20))
p = sns.heatmap(diamonds.corr(), annot=True,square=True)

In [ ]:
plt.figure(figsize=(10, 10))
j = sns.regplot(diamonds.clarity,diamonds.price, scatter=False)
axes = j.axes


In [ ]:
diamonds.clarity